In [3]:
import numpy as np

alpha=0.3 # capital share in production function
beta=0.6 # discount rate
A=20 # technology parameter in production function
K_max=10 # maximum of capital
grid=5 # numbers of grid points
max_iter=200 # maximum of iteration
tol=1e-4 # tolerance
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]
# interpolate capital uniformly to the grid points


def bellman(v, grid, beta):
    """
    bellman operator which transforms initial function v to Tv according to:
    
    Tv(k)=\max _{k'} \ln (A k^\alpha - k') +\beta* v(k')
    
    subject to:
    0\leq k' \leq A k^\alpha
    and 
    k' \in K
    ------
    parameters:
    v(k): initial value function
            ndarray, 1 \times grid
    grid: numbers of grids
            integer
    beta: discount factor
            float(dtype=32), 0<beta<1
    
    output:
    Tv(k): updated value function
            ndarray, 1 \times grid
    k'(k): control variable associated with Tv
            ndarray, 1 \times grid
    """
    # first, calculate the pseudo_v function:
    # \tilde v(k, k')=\ln (A*k^\alpha - k') +beta* v(k')
    #with the constraint: 
    # 0\leq k' \leq A*k^\alpha
    pseudo_v=np.zeros([grid, grid])
    for i in range(grid):
        k_prime_max=A*K[i]**alpha
        for j in range(grid):
            if K[j]<k_prime_max:
                pseudo_v[i, j]=np.log(A*K[i]**alpha-K[j])+beta*v[j]
            else:
                pseudo_v[i, j]=pseudo_v[i, j-1]
    # next, maximize pseudo_v row by row to get Tv(k) and k'(k)
    new_v=np.zeros(grid)
    K_prime=np.zeros(grid)
    K_prime_index=np.argmax(pseudo_v, axis=1)
    for i in range(grid):
        new_v[i]=max(pseudo_v[i])
        K_prime[i]=K[K_prime_index[i]]
    return new_v, K_prime

def update(v, N, grid, beta):
    """
    updates v according to the bellman operator for N times
    ------
    output:
    V: series of value functions
    ndarray, (N+1) \times grid
    K_p: series of control variables
    ndarray, N \times grid
    """
    V=np.zeros([N+1, grid])
    K_p=np.zeros([N, grid])
    V[0]=v
    for n in range(1, N+1):
        V[n], K_p[n-1]=bellman(V[n-1])
    return V, K_p

def vfi(v, grid, beta):
    """
    value function iteration until convergence or max_iter attained
    ------
    output:
    n: numbers of iteration
    integer
    V[n]: optimal value function
    ndarray, 1 \times grid
    K_p[n-1]: optimal policy function
    ndarray, 1 \times grid
    """
    V=np.zeros([max_iter+1, grid])
    K_p=np.zeros([max_iter, grid])
    V[0]=v
    n=1
    while n<=max_iter+1:
        V[n], K_p[n-1]=bellman(V[n-1])
        if max(V[n]-V[n-1])<=tol:
            return n, V[n], K_p[n-1]
            break
        else:
            n=n+1

# saving the optimal results
v_0=np.zeros(grid)
sigma=[K, vfi(v_0, grid, beta)[2]]

def next_k(k):
    """
    calculate next period capital according to the optimal policy function
    """
    return sigma[1][np.where(abs(K-k)<0.01)]

def c_k_path(k, T):
    """
    derive the time path for consumption and capital for T periods
    """
    cc=np.zeros(T)
    kk=np.zeros(T)
    kk[0]=k
    for t in range(1, T):
        kk[t]=next_k(kk[t-1])
        cc[t-1]=np.log(A*kk[t-1]**alpha-kk[t])
    return cc, kk

TypeError: bellman() missing 2 required positional arguments: 'grid' and 'beta'

In [1]:
#problem 4
update(v_0, grid, beta)

NameError: name 'update' is not defined

In [3]:
#problem 5
print(V_opt, K_prime_opt)

[ 7.94745286  8.20182354  8.35133401  8.4550566   8.53576161] [ 4.  6.  6.  6.  8.]


In [4]:
#problem 6
#change grid to 200
grid=200
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]


print(V_opt, K_prime_opt)

[ 6.60003102  6.8536376   7.0019742   7.10721641  7.1888524   7.25556879
  7.31195555  7.36081457  7.40390534  7.44244885  7.47732624  7.50915935
  7.53844088  7.56555293  7.59079876  7.61440539  7.63658416  7.65749977
  7.67728133  7.69604816  7.71389841  7.73091717  7.74718111  7.76275031
  7.7776861   7.79203436  7.80584137  7.81914542  7.83198134  7.84438625
  7.85638564  7.86800034  7.87925622  7.89017671  7.90078513  7.91109034
  7.92111305  7.93087121  7.94037537  7.94963598  7.95867051  7.96748881
  7.97609364  7.98450648  7.99272904  8.00076755  8.00863871  8.01633842
  8.02388355  8.03127594  8.03851977  8.04562518  8.05259124  8.05943268
  8.0661435   8.0727387   8.0792111   8.08557625  8.0918281   8.097979
  8.10402497  8.10997587  8.11582669  8.12159047  8.1272604   8.13284873
  8.13834754  8.14377091  8.14911006  8.15437494  8.15956369  8.16468183
  8.16972864  8.17470521  8.17961785  8.18446117  8.18924508  8.19396492
  8.19862613  8.20322881  8.20777181  8.2122632   8.2

In [5]:
#problem 7
c_k_path(7, 100)

(array([ 3.38119191,  3.35641936,  3.34955974,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.34664412,  3.34664412,
         3.34664412,  3.34664412,  3.34664412,  3.3

In [6]:
#problem 8
#change beta to 0.75

alpha=0.3 # capital share in production function
beta=0.75 # discount rate
A=20 # technology parameter in production function
K_max=10 # maximum of capital
grid=200 # numbers of grid points
max_iter=200 # maximum of iteration
tol=1e-4 # tolerance
K=np.linspace(K_max, 0, grid, endpoint=False)[::-1]
# interpolate capital uniformly to the grid points


def bellman(v):
    """
    bellman operator which transforms initial function v to Tv according to:
    
    Tv(k)=\max _{k'} \ln (A*k^\alpha - k') +beta* v(k')
    
    subject to:
    0\leq k' \leq A*k^\alpha
    and 
    k' \in K
    ------
    parameters:
    v(k): initial value function
    ndarray, 1 \times grid
    
    output:
    Tv(k): updated value function
    ndarray, 1 \times grid
    k'(k): control variable associated with Tv
    ndarray, 1 \times grid
    """
    # first, calculate the pseudo_v function:
    # \tilde v(k, k')=\ln (A*k^\alpha - k') +beta* v(k')
    #with the constraint: 
    # 0\leq k' \leq A*k^\alpha
    pseudo_v=np.zeros([grid, grid])
    for i in range(grid):
        k_prime_max=A*K[i]**alpha
        for j in range(grid):
            if K[j]<k_prime_max:
                pseudo_v[i, j]=np.log(A*K[i]**alpha-K[j])+beta*v[j]
            else:
                pseudo_v[i, j]=pseudo_v[i, j-1]
    # next, maximize pseudo_v row by row to get Tv(k) and k'(k)
    new_v=np.zeros(grid)
    K_prime=np.zeros(grid)
    K_prime_index=np.argmax(pseudo_v, axis=1)
    for i in range(grid):
        new_v[i]=max(pseudo_v[i])
        K_prime[i]=K[K_prime_index[i]]
    return new_v, K_prime

def update(v, N):
    """
    updates v according to the bellman operator for N times
    ------
    output:
    V: series of value functions
    ndarray, (N+1) \times grid
    K_p: series of control variables
    ndarray, N \times grid
    """
    V=np.zeros([N+1, grid])
    K_p=np.zeros([N, grid])
    V[0]=v
    for n in range(1, N+1):
        V[n], K_p[n-1]=bellman(V[n-1])
    return V, K_p

def vfi(v):
    """
    value function iteration until convergence or max_iter attained
    ------
    output:
    n: numbers of iteration
    integer
    V[n]: optimal value function
    ndarray, 1 \times grid
    K_p[n-1]: optimal policy function
    ndarray, 1 \times grid
    """
    V=np.zeros([max_iter+1, grid])
    K_p=np.zeros([max_iter, grid])
    V[0]=v
    n=1
    while n<=max_iter+1:
        V[n], K_p[n-1]=bellman(V[n-1])
        if max(V[n]-V[n-1])<=tol:
            return n, V[n], K_p[n-1]
            break
        else:
            n=n+1

# saving the optimal results
v_0=np.zeros(grid)
iters, V_opt, K_prime_opt=vfi(v_0)
sigma=[K, K_prime_opt]

def next_k(k):
    """
    calculate next period capital according to the optimal policy function
    """
    return sigma[1][np.where(abs(K-k)<0.01)]

def c_k_path(k, T):
    """
    derive the time path for consumption and capital for T periods
    """
    cc=np.zeros(T)
    kk=np.zeros(T)
    kk[0]=k
    for t in range(1, T):
        kk[t]=next_k(kk[t-1])
        cc[t-1]=np.log(A*kk[t-1]**alpha-kk[t])
    return cc, kk

print(V_opt, K_prime_opt)

[ 11.55013611  11.81846851  11.97542413  12.08677053  12.17315405
  12.24373605  12.30340589  12.3550951   12.40069076  12.44147744
  12.47837054  12.51205272  12.54303891  12.57172525  12.59842706
  12.62341297  12.64687994  12.66900744  12.68993412  12.70979236
  12.72867725  12.74668328  12.7638936   12.7803687   12.79617063
  12.81135009  12.82595899  12.84003683  12.85362102  12.86674499
  12.87943825  12.89172722  12.90363946  12.91519358  12.92641474
  12.93732137  12.94792824  12.95825187  12.96830767  12.97810631
  12.98766474  12.99699209  13.00610218  13.01500165  13.02370064
  13.03220575  13.04053239  13.04868473  13.0566652   13.06448338
  13.07215167  13.07966919  13.08703929  13.09427666  13.10138071
  13.10835336  13.11520648  13.12193854  13.12855401  13.13506272
  13.14146006  13.14775353  13.15394935  13.16004396  13.16604724
  13.1719559   13.17777629  13.18351294  13.18916148  13.19473402
  13.20022189  13.20563719  13.21097691  13.21624374  13.22143978
  13.22656

In [7]:
#finally, the time path
c_k_path(7, 100)

(array([ 3.32524461,  3.36699042,  3.3800286 ,  3.38290805,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.38517773,  3.38517773,
         3.38517773,  3.38517773,  3.38517773,  3.3